In [12]:
import json
import minsearch
import openai
import os
from openai import OpenAI

In [2]:
with open('/root/practice/logos/data/kjv.json', 'r') as f:
    bible_json = json.load(f)

In [3]:
with open('/root/practice/logos/data/video_data.json', 'r') as f:
    video_json = json.load(f)

### The json file contains some meta data i do not think we would be needing i would use `data["verse"]` to extract the relevant part of the json file 

In [4]:
documents = bible_json["verses"]

In [5]:
for document in documents:
    for key, value in document.items():
        document[key] = str(value)

In [6]:
index = minsearch.Index(
    text_fields = ["book", "chapter", "verse", "text"],
    keyword_fields = ["book_name"]
)

In [7]:
# Now pass it to index.fit
index.fit(documents)

In [8]:
boost = {"text": 5.0, "book" : 3.0}

q = "who is jesus"

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 10
)

In [ ]:
client = OpenAI()

In [14]:
def search(query):
    boost = {"text": 5.0, "book_name" : 3.0}
    
    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 10 )
    return results

In [15]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + f"Book: {doc['book_name']}\n Chapter: {doc['chapter']}\n Verse: {doc['verse']}\n Text: {doc['text']}\n"
        
    prompt = f"""
You are a knowledgeable Bible study assistant called "Logos" 
Your primary function is to provide accurate, insightful responses to users' questions about the Bible, offering contextual meaning for each chapter and verse. 
Answer the QUESTION based on the CONTEXT. use only the facts from the CONTEXT when answering the question.
If the CONTEXT doesn't contain the answer, output NONE
Be freindly like an assistant. you do not have to indicate that a context was given in your response

Your tasks include:

1. Answering Questions: When a user asks a question related to the Bible, respond with a clear and concise answer that is rooted in scripture. Provide relevant Bible verses to support your answer.

2. Contextual Understanding: For any chapter or verse referenced, explain its significance in the context of the broader narrative of the Bible. This includes historical, cultural, and theological insights.

3. Spiritual Guidance: Offer practical applications of the scriptures to help users navigate life situations. This could include advice based on biblical principles or encouragement through scripture.

4. Clarity and Compassion: Ensure that your responses are respectful, compassionate, and non-judgmental. Aim to foster a learning environment that encourages curiosity and spiritual growth.

5. Interactive Engagement: Encourage users to ask follow-up questions or seek clarification on topics they find challenging. Be patient and supportive in guiding them through their inquiries.

---


QUESTION: {query}

CONTEXT: {context}

""".strip()

    return prompt

In [118]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [133]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return answer

In [138]:
rag("What is acceleration")

'NONE'